In [11]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util

warnings.filterwarnings("ignore")

In [ ]:
load_dotenv()

HOPSWORK_API_KEY = os.getenv("HOPSWORK_API_KEY")
proj = hopsworks.login()
util.purge_project(proj)